### Problem 1

미국 S&P500지수를 추종 ETF를 비롯한 ETF 티커가 다음과 같이 준비되어 있습니다.

|  티커  | 종목                                                                                                      | 설명                   |
|:----:|:--------------------------------------------------------------------------------------------------------|:---------------------|
| SPY  | SPDR S&P 500 ETF Trust                                                                                  | SPX지수 추종 주식 ETF      |
| GOVT | iShares U.S. Treasury Bond ETF                                                                    | T-BOND 금리 추종 채권 ETF  |
| GLD  | SPDR Gold Trust                                                                                         | 금 가격 추종 ETF          |
| LQD  | iShares iBOXX $ Investment Grade Corporate Bond ETF                                                     | 선진국 투자등급(IG) 회사채 ETF |
| VGT  | Vanguard Information Technology Index Fund ETF | 미 증시 상장 기술주 ETF      |

초기 투자금이 1000달러라고 가정할 때 블랙리터만 모델을 기반으로 하는 EMP전략을 개발하려고 합니다. 예를들어 1000달러의 투자금이 있다면 2014년 1월 첫 거래일에는 각 ETF에 1000달러(각 종목은 시가총액가중 비중 조절)가 들어갑니다. 투자자의 전망이 아래와 같고, 전망에 대한 불확실성이 각각 아래와 같을 때, Black Litterman 모형을 활용하여 배분을 수행하고, 이를 기반으로 EMP전략을 개발하려고 합니다. 

- GLD의 연간 수익률이 10%일 것으로 예상하고, 이 전망에 대한 불확실성(표준편차)는 0.2이다.
- VGT의 연간 수익률이 SPY의 연간수익률보다 5%가 더욱 높을 것이며, 이 전망에 대한 불확실성(표준편차)는 0.1이다.

아래의 코드에서 잘못된 부분을 찾아서 수정한 뒤, 수정한 코드에는 주석을 달아주세요. 단, 아래의 가정을 따릅니다.

- 전날 종가를 기준으로 구매한다고 가정합니다.
- 종가는 yahoo finance의 Close를 사용합니다.
- 거래에 따른 슬리피지, 거래비용은 없다고 가정합니다.
- 유동성 리스크는 일단 고려하지 않습니다.
- 현금 보유는 고려하지 않습니다.
- 위험조정 상수는 1로 가정합니다
- 모든 종목은 Fractional Share(소수점 단위 소유)가 가능하다고 가정합니다.
- 시가 총액은 현재 시점을 기준으로 사용한다고 가정합니다. ETF의 경우 AUM을 사용하시면 됩니다.
- Market view matrix를 위한 가격 데이터는 직전 1년간의 공분산과 수익률 평균을 사용한다고 가정합니다.
- 2023년 1년간의 데이터를 사용하여 파라미터를 추정한 뒤, 2024년 1월부터 2024년 7월까지의 성과를 보고하세요.

In [42]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns

def get_markowitz_weights(ret, cov) :
    number_of_asset = ret.shape[0]
    init_guess = np.repeat(1 / number_of_asset, number_of_asset)
    bounds = ((0, 1), ) * number_of_asset
    
    constraints = {
        'type' : 'eq',
        'fun' : lambda weights : np.sum(weights) -1
    }
    
    def neg_sharpe(weights, ret, cov) :
        r = weights.T @ ret
        vol = np.sqrt(weights.T @ cov @ weights)
        return -r/vol
    
    res = minimize(
        neg_sharpe,
        init_guess,
        args = (ret, cov),
        constraints = constraints,
        bounds = bounds
    )
    
    return res.x

TICKERS = ['SPY','GOVT','GLD','LQD','VGT']
START_DATE = '2023-01-01'
END_DATE = pd.to_datetime('today')
ETF_TICKERS = yf.Tickers(TICKERS)
CAP_LIST = pd.Series(dtype = float)
for ticker in TICKERS :
    CAP_LIST.loc[ticker] = yf.Ticker(ticker).info['totalAssets']

data = yf.download(
    TICKERS,
    start = START_DATE,
    end=END_DATE,
    interval='1d',
    progress = False
)['Close']

market_weights = CAP_LIST / CAP_LIST.sum()
ret_vector = data.loc['2023'].pct_change().dropna().mean() * 252
cov_mat = data.loc['2023'].pct_change().dropna().cov() * 252 # covariance 할때 *252로 알고 있습니다. 

market_port_ret = ret_vector.multiply(market_weights).sum()
market_port_var = np.matmul(
    market_weights.T, np.matmul(
        cov_mat, market_weights
    )
)
market_lambda = market_port_ret / np.sqrt(market_port_var)
pi = pd.Series(
    np.dot(np.dot(market_lambda, cov_mat), market_weights),
    index = TICKERS
)

invester_ret = np.array(
    [0.1, 0.05]
)
invester_mat = np.array(
    [
        [0, 0, 1, 0, 0],
        [-1, 0, 0, 0, 1]
    ]
) # vgt -spy = 0.05 여야합니다. 
tau = 1 # 위험조정상수 1로 고정 

omega = np.diag([0.04, 0.01]) # 조건에 설정된 대로 0.2^2, 0,1^2로 설정

term1 = np.linalg.inv(np.dot(tau, cov_mat))
term2 = np.dot(np.dot(np.transpose(invester_mat), np.linalg.inv(omega)), invester_mat)
term3 = np.dot(np.linalg.inv(np.dot(tau, cov_mat)), pi)
term4 = np.dot(np.dot(np.transpose(invester_mat), np.linalg.inv(omega)), invester_ret)

post_cov_mat = np.linalg.inv(term1 + term2)
post_ret_vec = np.dot(post_cov_mat, term3 + term4)

bl_weights = get_markowitz_weights(post_ret_vec, post_cov_mat)
bl_ret = np.dot(bl_weights, post_ret_vec)
bl_vol = np.sqrt(np.dot(bl_weights.T, np.dot(post_cov_mat, bl_weights)))

# black litterman portfolio
bl_portfolio = bl_weights * data.loc['2024'].pct_change().dropna()

# 포트폴리오의 수익률 계산
bl_portfolio_returns = bl_portfolio.sum(axis=1)

# 2024년 1월부터 7월까지의 수익률과 변동성 계산
bl_return_2024 = bl_portfolio_returns.mean() * len(bl_portfolio_returns)
bl_volatility_2024 = bl_portfolio_returns.std() * np.sqrt(len(bl_portfolio_returns))

print(f"Black-Litterman 2024 Jan-Jul Return: {bl_return_2024}")
print(f"Black-Litterman 2024 Jan-Jul Volatility: {bl_volatility_2024}")


Black-Litterman 2024 Jan-Jul Return: 0.10487925817461541
Black-Litterman 2024 Jan-Jul Volatility: 0.10080623682199992


C:\Users\ted86\AppData\Local\Temp\ipykernel_205716\1890365557.py:55: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  market_weights.T, np.matmul(


### Problem 2

동일한 자산군에 대해서 Markowitz Optimization을 수행한 뒤, 1번의 포트폴리오와 비교해 2024년간 포트폴리오 수익률과 자산 배분 비중을 비교하세요. Markowitz Optimization과 Black Litterman Optimization을 비교하였을 때, 성과가 좋은 배분 방법은 무엇인가요?

In [43]:
# Markowitz Optimization
markowitz_weights = get_markowitz_weights(ret_vector, cov_mat)
markowitz_ret = np.dot(markowitz_weights, ret_vector)
markowitz_vol = np.sqrt(np.dot(markowitz_weights.T, np.dot(cov_mat, markowitz_weights)))
post_cov_mat = np.linalg.inv(term1 + term2)
post_ret_vec = np.dot(post_cov_mat, term3 + term4)

# 마코위츠
markowitz_portfolio = markowitz_weights * data.loc['2024'].pct_change().dropna()

# 포트폴리오의 수익률 계산
markowitz_portfolio_returns = markowitz_portfolio.sum(axis=1)

# 2024년 1월부터 7월까지의 수익률과 변동성 계산
mark_return_2024 = markowitz_portfolio_returns.mean() * len(markowitz_portfolio_returns)
mark_volatility_2024 = markowitz_portfolio_returns.std() * np.sqrt(len(markowitz_portfolio_returns))

print(f"Markowitz 2024 Jan-Jul Return: {markowitz_return_2024}")
print(f"Markowitz 2024 Jan-Jul Volatility: {markowitz_volatility_2024}")
print(f"Black-Litterman 2024 Jan-Jul Return: {bl_return_2024}")
print(f"Black-Litterman 2024 Jan-Jul Volatility: {bl_volatility_2024}")
print('성과로만 판단한다면 Markowitz가 더 좋은 배분 방법입니다')

Markowitz 2024 Jan-Jul Return: 0.2010638993513701
Markowitz 2024 Jan-Jul Volatility: 0.10358253712219424
Black-Litterman 2024 Jan-Jul Return: 0.10487925817461541
Black-Litterman 2024 Jan-Jul Volatility: 0.10080623682199992
성과로만 판단한다면 Markowitz가 더 좋은 배분 방법입니다


### Problem 3

1번과 2번에서 구한 포트폴리오를 이용해 연율화된 Sharpe Ratio, MDD, CAGR을 구하고, 두 포트폴리오의 성과 통계량을 비교하여 값을 보고하세요. 단, 무위험이자율은 연간 5.25%로 가정하고, 시장 수익률은 S&P500지수의 수익률을 이용합니다.

In [44]:
def calculate_cagr(cumulative_returns, periods_per_year=252):
    n_periods = len(cumulative_returns) / periods_per_year
    ending_value = cumulative_returns.iloc[-1]
    cagr = (1 + ending_value) ** (1 / n_periods) - 1
    return cagr

def calculate_mdd(cumulative_returns):
    drawdown = cumulative_returns / cumulative_returns.cummax() - 1
    mdd = drawdown.min()
    return mdd

def calculate_sharpe_ratio(daily_returns, risk_free_rate=0.0525, periods_per_year=252):
    excess_returns = daily_returns - risk_free_rate / periods_per_year
    annualized_sharpe_ratio = np.sqrt(periods_per_year) * excess_returns.mean() / excess_returns.std()
    return annualized_sharpe_ratio

# 연율화된 Sharpe Ratio 
markowitz_sharpe_ratio = calculate_sharpe_ratio(markowitz_portfolio_returns)
bl_sharpe_ratio = calculate_sharpe_ratio(bl_portfolio_returns)

# MDD 
markowitz_cumulative_returns = (1 + markowitz_portfolio_returns).cumprod() - 1
bl_cumulative_returns = (1 + bl_portfolio_returns).cumprod() - 1

markowitz_mdd = calculate_mdd(markowitz_cumulative_returns)
bl_mdd = calculate_mdd(bl_cumulative_returns)

# CAGR 
markowitz_cagr = calculate_cagr(markowitz_cumulative_returns, periods_per_year=252)
bl_cagr = calculate_cagr(bl_cumulative_returns, periods_per_year=252)

# 결과 
print("Markowitz Portfolio:")
print(f"Sharpe Ratio: {markowitz_sharpe_ratio}")
print(f"MDD: {markowitz_mdd}")
print(f"CAGR: {markowitz_cagr}")
print("\nBlack-Litterman Portfolio:")
print(f"Sharpe Ratio: {bl_sharpe_ratio}")
print(f"MDD: {bl_mdd}")
print(f"CAGR: {bl_cagr}")

Markowitz Portfolio:
Sharpe Ratio: 1.5969903638191076
MDD: -0.793888374469682
CAGR: 0.3148455855858081

Black-Litterman Portfolio:
Sharpe Ratio: 1.0013243020522153
MDD: -1.0165117083921391
CAGR: 0.1952525042349773
